<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.2: Generators: Collections 生成器：集合
**Prev:上一个小节： [Generators: Parameters 生成器：变量](3.1_parameters.ipynb)**<br>
**Next:下一个小节： [Interlude: Chisel Standard Library  间歇：Chisel 标准库](3.2_interlude.ipynb)**


## Motivation 动机
Generators will frequently have to deal with variable numbers of objects, whether they are IOs, modules, or test vectors. 生成器会经常处理很多对象，不论是 IO，模块或者是测试向量。
Collections are important building blocks for dealing with situations like this. 集合可以生成专为很好地处理这些情况的代码块。
This module will introduce Scala collections and how to use them with Chisel generators. 这个单元将会介绍 Scala 集合并且它在 Chisel 生成器中的使用方法。

## Setup 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Note we add a new import here because `mutable.ArrayBuffer` lives in `scala.collections`. 注意我们添加了一个新的引用，因为 `mutable.ArrayBuffer` 包含在 `scala.collections` 中。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import scala.collection._

---
# Generators and Collections 生成器与集合<a name="generators-and-collections"></a> 
In this section, we will focus on the concept of *generators* and the use of Scala collections as a tool to implement them. 在这个小节中，我们将会专注于*生成器*的概念，并且学习怎样把 Scala 集合作为实现它的工具。
Instead of looking at Chisel code as an *instance* of a circuit, i.e. a description of a particular circuit, 我们通常不把 Chisel 代码看作一个电路的实例，比如说描述一个特定的电路，
we will instead consider it to be a generator of a circuit. 而将它看作是某个电路的生成器。

We will start by considering the FIR filter from previous exercises.  我们将从之前练习中的 FIR 滤波器开始讲解。

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


This circuit is a simple case of a generator because it can generate versions of this 4-tap filter with different coefficients.  这个电路是一个生成器的简单例子，因为它可以生成带有不同系数的多版本 4 抽头滤波器。
But what if we want the circuit to have more taps?  We are going to do this in several steps.
 但是如果我们想要这个电路有更多的抽头呢？我们需要完成接下来的步骤。
- Build a software *Golden Model* of a tap configurable FIR. 建立一个可配置抽头数量 FIR 滤波器的软件*黄金模型*。
- Redesign our test to use this model, and confirm that it works. 重新运用这个模型设计测试文件，以确保这个模型是无误的。
- Refactor our My4ElementFir to allow an configurable number of taps. 重构 MyManyElementFir 以可配置抽头数。
- Test the new circuit using our new test harness. 用新的测试包测试新生成的电路。

<span style="color:blue">**Example: FIR Golden Model 举例： FIR 黄金模型**</span><br><a name="fir-golden-model"></a> 
Below is a Scala software implementation of a FIR circuit.  下面是一个使用 Scala 语言实现的 FIR 电路。

In [ ]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps. 一个简单的实现带有任意数抽头的 FIR 滤波器的方法。
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

### Seq 系列
Note that `taps` has become a `Seq[Int]` which means that the user of the class can pass an arbitrarily-long sequence of `Int`s when constructing the class. 注意，`taps` 已经变成了 `Seq[Int]`，这表明类的使用者在构造类的时候可以传递任意长度 `Int` 序列。
### Registers 寄存器
With `  var pseudoRegisters = List.fill(taps.length)(0)` we create a `List` that will hold values from previous cycles.  `List` was chosen because its syntax of adding an element to the head and removing the last element is very simple. Just about any member of the scala collections family could be used.  We are also initializing this list to contain all zeros. 通过`  var pseudoRegisters = List.fill(taps.length)(0)`我们创建了一个将保留前一个循环的值的 `List`。因为 `List` 从头增加一个元素、从尾删去一个元素的语法很简单，所以选择使用它。任何 Scala 集合类型都可以使用。我们也初始化这个列表至全零。
### Poke 
Our class adds a poke function/method that emulates putting a new input into the filter and cycling the clock. 我们的类加入了一个 poke 函数/方法，它可以实现在滤波器中添加一个新的输入并且驱动时钟。
### Updating the registers 更新寄存器
The line `pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` first uses the `take` method of list to keep the all but the last element of the list, then uses the `::` list concatentation operator to add `value` to the head of the reduced version of the list. 这一行 `pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` 第一次使用了列表的 `take` 方法来保留除了最后一个元素之外的所有值，然后使用列表的级联操作符 `::` ，以在删去最后一位的列表头的位置，添加 `value`。
### Computing the output 计算输出
A simple for loop with an accumulator sums each element of the list times its corresponding tap coefficient. The line with just `accumulator` returns that value as the function result. 循环使用累加器对每一个列表中的元素乘上对应的抽头系数的积求和。一行只有 `accumulator` 的返回这个函数的结果。
## Adapting our previous test for testing our golden model 改编我们早期的测试以测试我们的黄金模型
We will now use our previous work to confirm that our golden model works.  A bit of editing magic takes our previous tests harness and morphs it into... 我们现在使用我们早先的设计确保我们的黄金模型正确。有一些编辑的魔力发生在了早期的测试包中，然后将它改变成了……

In [ ]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

Executing the previous block demonstrates that our software model returns the same results as My4ElementFir did. 执行上一个代码块显示了我们的软件模型返回了和 My4ElementFir 一样的结果。


## Test circuit using the golden model. 使用黄金模型测试电路<a name="use-golden-model-as-test"></a> 
Now that we are reasonably confident about our golden model, we re-write our test to compare the circuit outputs with the output of the golden model, instead of using laboriously hand-crafted examples. 既然我们已经有理由对我们的黄金模型很自信，那么我们重写我们的测试文件来对比这个电路的输出和黄金模型的输出。
What follows is a quick first pass to do it. 下面是一个无误、快速实现它的方式。

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

Our test runs for 100 cycles, and checks that the two different methods, hardware and software, are in sync at each step. 我们硬件和软件测试，两个不同的检查方法，同步运行100个循环。

### Things to watch out for 需要注意的事情
(i.e., mistakes we actually committed while writing this. 也即是我们在编程的时候实际犯过的错误。)

1. Getting the step in the right place. Software and hardware execute differently; it's easy to get this wrong. 在软件、硬件部分执行对应的步骤。软件和硬件执行方式不同，这很容易弄错。
1. This test is weak because it is very sensitive to how the IOs and registers are sized. Implementing a software golden model that observes wrapping behavior at arbitrary data bit widths can be complicated.  Here we just make sure that we only pass in values that fit. 这个测试是比较脆弱的，因为它对 IO 和寄存器定义大小的方式是敏感的。我们编写的这个 Chisel 代码相当于一个任意位宽的 wrapper，而如果要实现一个检测这样任意位宽 wrapper 的黄金模型程序，这将是非常复杂的。在这里我们就确保只传递合适的值。

<span style="color:blue">**Example: Parameterized FIR Generator 举例：参数化的 FIR 生成器**</span><br><a name="fir-golden-model"></a> 
Below we have created a new Filter class, `MyManyElementsFilter` that takes a `Seq` of constants to use for taps.  This list can be any number of elements.下面我们创建了一个新的滤波器的类，`MyManyElementsFilter` 。它使用了一个常数的 `Seq` 来定义抽头的位数。
For good measure a `bitWidth` has been added that allows us to control the sizes of numbers that can be handled by our circuit.  新添加的 `bitWidth` 参数使得电路处理的位宽可以被我们控制。
In response the variable length we have had to refactor the creation of registers and how they are connected. 对应的，我们已经加入了可变化的长度，以重构寄存器的声明和连接。
The methodology used below uses a simple subset of the available library of collection functions. 下面的这个方法使用了集合函数的库中一个简单的子集
Later sections show how to more succinctly express the behavior in a way that also makes what is happening clearer. 后续的小节会展示怎样更加简洁地、清晰地表述电路的行为。

In [ ]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

#### How we did it 我们如何实现的
There are three parallel sections starting at lines 7, 13, and 18. 从7、13、18行开始，有三个平行的小节。
We are using a Scala collection type called `ArrayBuffer`. 我们使用了一个 Scala 集合类型——`ArrayBuffer`。
`ArrayBuffer` allows you to append elements using the `+=` operator (also insert and delete, but we don't need this).    `ArrayBuffer` 允许你使用 `+=` 操作符附加元素（也可以插入和删除，但是我们不需要）。
First, we create an ArrayBuffer `regs` whose elements will be `UInt`s. 首先，我们声明一个 ArrayBuffer `regs`，它的元素将会是 `UInt` 类型。
Then iterate over the taps, adding the input as the first element followed by creating registers using RegNext which connect the input of the register to the previous element (`regs(i-1)`) and initializes it to unsigned zero (`0.U`).  然后反复添加抽头，首先输入作为第一个元素，然后是使用 RegNext 声明寄存器，并且也将该寄存器的输入和上一个元素连接起来 (`regs(i-1)`)，最后将其初值设置为无符号零 (`0.U`)。
These registers will hold the previous values of inputs as they are needed. 这些寄存器将会暂存它们需要的输入的先前值。

Next, we create another ArrayBuffer `muls` of `UInt`s. 然后，我们声明了另外一个 `UInt` 类型的 ArrayBuffer `muls`
Each element of muls will be a node whose i-th element is the product of the `regs(i)` and `const(i)`. `muls` 的每一个元素都是一个结点，它的第 i 个元素是 `regs(i)` 与 `const(i)`的积。

Note the use of the `scan.last` method. 注意 `scan.last` 方法的使用。
It takes the last element of a collection, and is a more elegant alternative to `regs(i - 1)` used during the `regs` construction. 它获取了一个集合的最后一个元素，是比在构建 `regs` 使用的 `regs(i - 1)` 更加优雅的替换方法。

### Does it behave the same as `My4ElementFir`? 它与 `My4ElementFir` 功能一样吗？
A good first test of our new version is to see if it can pass the test we just applied to the
`My4ElementFir`. 我们新版本的初步测试主要是为了检查它是否可以通过测试 `My4ElementFir` 的测试程序。
We create an instance of `MyManyElementFir` and run even more data through it. 我们编写了一个 `My4ElementFir` 的实例，并且通过更多数据测试它。

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1), 8)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

### Now let's test a bunch of different sized FIR filters 现在，我们测试一串不同大小的 FIR 滤波器。
We create some helper functions: `r` which gets a random number; `runOneTest` which creates a golden model and a hardware simulation of a filter for a particular set of taps, and then runs at least twice the number of taps worth of data through the filter. 我们新加入了一些更加有帮助的函数：通过 `r` 获得一个随机数；通过 `runOneTest` 创建一个黄金模型和针对特定数量抽头的滤波器硬件激励，运行的时候至少使两倍于抽头数量的测试数据通过滤波器。

In [ ]:
/** a convenience method to get a random integer 一个可以获得随机整数的方便方法
  */
def r(): Int = {
  scala.util.Random.nextInt(1024)
}

/**
  * run a test comparing software and hardware filters 运行一个对比软件和硬件滤波器的测试
  * run for at least twice as many samples as taps 至少运行两倍于抽头数量的测试样本
  */
def runOneTest(taps: Seq[Int]) {
  val goldenModel = new ScalaFirFilter(taps)

  Driver(() => new MyManyElementFir(taps, 32)) {
    c => new PeekPokeTester(c) {
      for(i <- 0 until 2 * taps.length) {
        val input = r()

        val goldenModelResult = goldenModel.poke(input)

        poke(c.io.in, input)

        expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

        step(1)
      }
    }
  }
}

for(tapSize <- 2 until 100 by 10) {
  val taps = Seq.fill(tapSize)(r())  // create a sequence of random coefficients 声明一个随机系数序列

  runOneTest(taps)
}

### Just for fun, let's make a bigger one 让我们玩一个更大的
The following will run a single test on a 500 tap FIR filter.下面我们将运行一个拥有 500 抽头 FIR 滤波器的单个测试。
It can take a minute or so to run. 运行将会花费一分钟左右的时间。
(Hint: Watch for the Scala ● to change to Scala ○ on the Toolbar when the execution completes.)（提示：注意在工具栏的 `Scala ●` 将在运行完成的时候变成 `Scala ○`）

In [ ]:
runOneTest(Seq.fill(500)(r()))

In [ ]:
val taps = Seq.fill(500)(r())

val goldenModel = new ScalaFirFilter(taps)

Driver(() => new MyManyElementFir(taps, 32)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = r()

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

---
# Hardware Collections 硬件集合

<span style="color:blue">**Example: Add run-time configurable taps to our FIR 举例：为 FIR 添加运行时可配置的抽头**</span><br>
The following code adds an additional `consts` vector to the IO of our FIR generator which allows the coefficients to be changed externally after circuit generation. 下面的代码给 FIR 生成器的 IO 添加了一个附加的 `consts` 向量，这允许系数在电路生成后从外部是可变的
This is done with the Chisel collection type `Vec`. 这使用 Chisel 集合类型—— `Vec` 实现的。
`Vec` supports many of the scala collection methods but it can only contain Chisel hardware elements. `Vec` 支持很多的 Scala 集合的方法但是它只能包含 Chisel 硬件元素。
`Vec` should only be used in situations where ordinary Scala collections won't work. `Vec` 只应该在普通的 Scala 集合不能起效的时候才使用。 
Basically this is in one of two situations. 基本上是以下两种情况之一。
1. You need a collection of elements in a Bundle, typically a Bundle that will be used as IO. 你需要在 Bundle 中使用集合，特别是将作为 IO 使用的 Bundle。 
1. You need to access the collection via an index that is part of the hardware (think Register File). 你需要通过硬件查找表获取集合（比如说寄存器蒲） 


In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(VecInit(Seq.fill(length - 1)(0.U(8.W))))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyDynamicElementVecFir(4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

<span style="color:red">**Exercise: 32-bit RISC-V Processor 练习：32 位 RISC-V 处理器**</span><br><a name="fir-golden-model"></a>

A [register file](https://en.wikipedia.org/wiki/Register_file) is an important building block for making a processor. [寄存器蒲](https://en.wikipedia.org/wiki/Register_file)是组成处理器的一个很重要部件。
A register file is an array of registers that can be read from or written to via a number of read or write ports. 寄存器蒲是一个寄存器阵列，这些寄存器可以通过读写端口被读取或者写入。
Each port consists of an address and data field. 每一个端口都包含了地址和数据部分。

The [RISC-V instruction set architecture](https://riscv.org/specifications/) defines several variants, the simplest of which is called RV32I. [RISC-V 指令集架构](https://riscv.org/specifications/) 定义了一些变体，其中最简单的叫 RV32I。
RV32I has a size-32 array of 32-bit registers. RV32I 有一个由 32 个 32 位寄存器组成的阵列。
**The register at index 0 (the first register) is always zero when you read from it, regardless of what you write to it** (it's often useful to have 0 handy). **在索引 0 的寄存器（第一个寄存器）在你读的时候永远是零值，不管你给它写入的什么**（要方便地获得 0 时它就很有帮助了。）

Implement a register file for RV32I with a single write port and a paramterized number of read ports. 实现一个带有一个写端口和可配置数量的读端口的 RV32I 寄存器蒲。
Writes will only be performed when `wen` (write enable) is asserted. 在 `wen` （写使能）有效时才可以写入。

In [ ]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // A Register of a vector of UInts 一个无符号整数向量寄存器
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))

}

In [ ]:
chisel3.iotesters.Driver(() => new RegisterFile(2) ) { c => new PeekPokeTester(c) {
    def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
        poke(c.io.raddr(port), addr)
        expect(c.io.rdata(port), value)
    }
    def write(addr: Int, value: Int): Unit = {
        poke(c.io.wen, 1)
        poke(c.io.wdata, value)
        poke(c.io.waddr, addr)
        step(1)
        poke(c.io.wen, 0)
    }
    // everything should be 0 on init 在开始时所有值都应该是零
    for (i <- 0 until 32) {
        readExpect(i, 0, port = 0)
        readExpect(i, 0, port = 1)
    }

    // write 5 * addr + 3 
    for (i <- 0 until 32) {
        write(i, 5 * i + 3)
    }

    // check that the writes worked 检查写是否有效
    for (i <- 0 until 32) {
        readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
    }

}}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    for (i &lt;- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }

</pre></article></div></section></div>

---
# You're done! 恭喜你，完成了本节内容的学习！

[Return to the top. 回到顶层。](#top)